In [1]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, concatenate, Flatten, Dense


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the input images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape the input images to 4D tensors
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# Define the Inception Module
def inception_module(x, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    conv3x3_reduce = Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    conv3x3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3_reduce)

    conv5x5_reduce = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(x)
    conv5x5 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5x5_reduce)

    maxpool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    maxpool_proj = Conv2D(filters[5], (1, 1), padding='same', activation='relu')(maxpool)

    inception_output = concatenate([conv1x1, conv3x3, conv5x5, maxpool_proj], axis=-1)
    return inception_output

In [4]:
# Define the GoogLeNet model
def googlenet(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # Initial Convolution and MaxPooling
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Inception Modules
    x = inception_module(x, [64, 64, 128, 32, 32, 32])
    x = inception_module(x, [128, 128, 192, 96, 96, 64])

    # Add more inception modules as needed

    # Main Classifier
    x = AveragePooling2D((7, 7), strides=(1, 1))(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=x)

    return model

# Create the GoogLeNet model for MNIST
input_shape = (28, 28, 1)
num_classes = 10
model = googlenet(input_shape, num_classes)

# Print a summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 14, 14, 64)           3200      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 7, 7, 64)             0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_2 (Conv2D)           (None, 7, 7, 64)             4160      ['max_pooling2d[0][0]']   

In [6]:
# Compile and train the model using the Adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=5, validation_data=(x_test, y_test))


Epoch 1/5
469/469 [==============================] - 395s 836ms/step - loss: 0.2014 - accuracy: 0.9398 - val_loss: 0.0905 - val_accuracy: 0.9729
Epoch 2/5
469/469 [==============================] - 387s 825ms/step - loss: 0.0808 - accuracy: 0.9747 - val_loss: 0.0591 - val_accuracy: 0.9802
Epoch 3/5
469/469 [==============================] - 383s 816ms/step - loss: 0.0560 - accuracy: 0.9823 - val_loss: 0.0323 - val_accuracy: 0.9895
Epoch 4/5
469/469 [==============================] - 382s 814ms/step - loss: 0.0418 - accuracy: 0.9869 - val_loss: 0.0401 - val_accuracy: 0.9865
Epoch 5/5
469/469 [==============================] - 380s 810ms/step - loss: 0.0381 - accuracy: 0.9884 - val_loss: 0.0344 - val_accuracy: 0.9885


In [7]:
# Evaluate the model and print the confusion matrix
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
confusion_mat = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(confusion_mat)

313/313 [==============================] - 17s 55ms/step
Confusion Matrix:
[[ 979    0    0    0    0    0    1    0    0    0]
 [   3 1130    1    1    0    0    0    0    0    0]
 [  12    2 1009    0    0    0    1    2    6    0]
 [   0    0    0  999    0    7    0    1    2    1]
 [   0    0    0    0  967    0    4    0    2    9]
 [   1    0    0    1    0  887    3    0    0    0]
 [   5    5    0    0    0    2  945    0    1    0]
 [   1    6    5    0    1    1    0 1005    0    9]
 [   4    0    0    0    0    1    2    0  966    1]
 [   2    0    0    0    3    1    0    2    3  998]]


In [8]:
# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

Precision: 0.9885921791938647
Recall: 0.9885
F1-Score: 0.988502524944222
